<a href="https://colab.research.google.com/github/anvy87/AlgoTrading/blob/master/kc_pivotprice_intraday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install kiteconnect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.5/771.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.8 MB/s eta 0:00:00


In [2]:
from kiteconnect import KiteConnect
import pandas as pd
import numpy as np
import logging
import os
import time
import datetime as dt
api_key = "1ll4p8ziqlk8hh7g"
api_secret = "x59r5u1vd25annfpgts22wv8ykfueirl"
kite = KiteConnect(api_key=api_key)

In [4]:
print(kite.login_url()) #use this url to manually login and authorize yourself

https://kite.zerodha.com/connect/login?api_key=1ll4p8ziqlk8hh7g&v=3


In [5]:
#generate trading session
request_token = "8BvyDbnoEXSLRkTd8vafz0ap9KD4dDwq" #Extract request token from the redirect url obtained after you authorize yourself by loggin in
data = kite.generate_session(request_token, api_secret=api_secret)
#create kite trading object
kite.set_access_token(data["access_token"])

In [6]:
#get dump of all NSE instruments
instrument_dump = kite.instruments("NSE")
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv("NSE_Instruments.csv",index=False)

In [7]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [32]:
def fetchOHLC(ticker,interval,duration):
  """extracts historcial data and outputs in the form of dataframe"""
  instrument = instrumentLookup(instrument_df,ticker)
  data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration),dt.date.today(),interval))
  # data1 = pd.DataFrame(kite.historical_data(instrument,dt.date.today(),dt.date.today(),interval))
  # print(data1.head())
  data.set_index("date",inplace=True)
  return data

In [9]:
# List of Nifty 50 constituent trading symbols (update as per latest list)
nifty50_symbols = ["RELIANCE", "HDFCBANK", "BHARTIARTL", "TCS", "ICICIBANK", "SBIN", "INFY", "HINDUNILVR", "BAJFINANCE", "ITC"]

In [10]:
instrument_token = [6401, 40193, 60417, 81153, 98049, 119553, 177665, 225537, 315393, 341249, 345089, 348929, 408065, 424961, 492033, 502785, 519937,
 633601, 738561, 779521, 857857, 878593, 884737, 895745, 897537, 969473, 1270529, 1304833, 1346049, 1510401, 1850625, 2815745, 2939649,
 2952193, 2953217, 2977281, 3001089, 3465729, 4267265, 4268801, 4598529, 4644609, 5215745, 5582849, 3861249]

# **placing order**

In [11]:
def placeMarketOrder(symbol, buy_sell, quantity):
  # place intraday market order on NSE
  if buy_sell == "buy":
    t_type = kite.TRANSACTION_TYPE_BUY
  elif buy_sell == "sell":
    t_type = kite.TRANSACTION_TYPE_SELL
  kite.place_order(tradingsymbol=symbol,
                   exchange=kite.EXCHANGE_NSE,
                   transaction_type=t_type,
                   quantity=5*quantity, # 5x leverage
                   order_type=kite.ORDER_TYPE_MARKET,
                   product=kite.PRODUCT_MIS,
                   variety=kite.VARIETY_REGULAR)

In [21]:
df.iloc[20, : ]['close']

np.float64(830.9)

In [36]:
df_yesterday = fetchOHLC("SBIN","5minute", 3)
print("SBIN", df_yesterday.head(30))
close_11am = df.iloc[20, : ]['close']
print(close_11am)
df_today = fetchOHLC("SBIN","5minute",0)
print(df_today.head())
intraday_high = df_today['high'].max()
print(intraday_high)
intraday_low = df_today['low'].min()
print(intraday_low)
capital = 10000 #position size
quantity = int(capital/df_today["close"][-1])
'''instrument = instrumentLookup(instrument_df, i)
if instrument != -1: # Check if instrument token was found
    yesterday_close = kite.quote([instrument])[str(instrument)]['ohlc']['close'] # Use instrument token as key
    #print(close_11am)
    #print(f"{i}: {yesterday_close}") # Print the symbol and its close price
else:
    print(f"Instrument token not found for {i}")'''
pivot_price = (intraday_high+intraday_low+yesterday_close)/3
#print(pivot_price, close_11am)
if close_11am > pivot_price:
  print(f"buy SBIN")
  #placeMarketOrder(i, "buy", quantity)
elif close_11am < pivot_price:
  print(f"sell SBIN")
  #placeMarketOrder(i, "sell", quantity)

SBIN                              open    high     low   close  volume
date                                                             
2025-08-22 09:15:00+05:30  824.00  826.00  824.00  825.05  130252
2025-08-22 09:20:00+05:30  825.20  825.40  824.50  824.70   66527
2025-08-22 09:25:00+05:30  824.75  824.75  823.30  823.80   96588
2025-08-22 09:30:00+05:30  823.65  823.80  821.80  822.10   93507
2025-08-22 09:35:00+05:30  822.25  822.95  822.00  822.65   71168
2025-08-22 09:40:00+05:30  822.80  823.25  822.60  822.75   32982
2025-08-22 09:45:00+05:30  822.75  822.90  822.35  822.50   39704
2025-08-22 09:50:00+05:30  822.45  823.00  822.40  822.55   24815
2025-08-22 09:55:00+05:30  822.50  822.50  821.50  821.85   66304
2025-08-22 10:00:00+05:30  821.90  821.90  820.60  821.15  106648
2025-08-22 10:05:00+05:30  821.20  822.55  821.05  822.10   33477
2025-08-22 10:10:00+05:30  822.15  822.70  822.05  822.70   29360
2025-08-22 10:15:00+05:30  822.60  822.70  821.70  821.80   38141
2025-

/tmp/ipython-input-3458887839.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quantity = int(capital/df_today["close"][-1])


In [13]:
for i in nifty50_symbols:
  df = fetchOHLC(i,"5minute", 4)
  #print(i, df)
  capital = 10000 #position size
  quantity = int(capital/df["close"][-1])
  close_11am = df.iloc[20, : ]['close']
  intraday_high = df['high'].max()
  intraday_low = df['low'].min()
  instrument = instrumentLookup(instrument_df, i)
  if instrument != -1: # Check if instrument token was found
      yesterday_close = kite.quote([instrument])[str(instrument)]['ohlc']['close'] # Use instrument token as key
      #print(close_11am)
      #print(f"{i}: {yesterday_close}") # Print the symbol and its close price
  else:
      print(f"Instrument token not found for {i}")
  pivot_price = (intraday_high+intraday_low+yesterday_close)/3
  #print(pivot_price, close_11am)
  if close_11am > pivot_price:
    print(f"buy {i}")
    #placeMarketOrder(i, "buy", quantity)
  elif close_11am < pivot_price:
    print(f"sell {i}")
    #placeMarketOrder(i, "sell", quantity)

/tmp/ipython-input-3467370738.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quantity = int(capital/df["close"][-1])


buy RELIANCE
buy HDFCBANK
buy BHARTIARTL
buy TCS
buy ICICIBANK
buy SBIN
buy INFY
buy HINDUNILVR
sell BAJFINANCE
buy ITC
